In [11]:
import yfinance as yf
import pandas as pd
import numpy as np

In [16]:
def download_data(ticker, start, end):
    return yf.download(ticker, start=start, end=end)

def calculate_bollinger_bands(data, window=20, num_sd=2):
    data['SMA'] = data['Close'].rolling(window=window).mean()
    data['std'] = data['Close'].rolling(window=window).std()
    data['Upper Band'] = data['SMA'] + (data['std'] * num_sd)
    data['Lower Band'] = data['SMA'] - (data['std'] * num_sd)
    return data

def calculate_adx(data, window=14):
    high = data['High']
    low = data['Low']
    close = data['Close']

    plus_dm = high.diff().clip(lower=0)
    minus_dm = -low.diff().clip(upper=0)

    tr = pd.concat([high - low, abs(high - close.shift(1)), abs(low - close.shift(1))], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()

    plus_di = 100 * (plus_dm.rolling(window=window).mean() / atr)
    minus_di = abs(100 * (minus_dm.rolling(window=window).mean() / atr))
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.rolling(window=window).mean()

    data['ADX'] = adx
    return data

def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

def apply_trading_strategy(data, initial_capital=100000):
    cash = initial_capital
    position = 0
    buy_signals = []
    sell_signals = []
    portfolio_value = []

    for i in range(len(data)):
        if data['Close'][i] < data['Lower Band'][i] and data['ADX'][i] > 20 and data['RSI'][i] < 30:
            if position == 0:  # Buy signal
                buy_price = data['Close'][i]
                position = cash // buy_price
                cash -= position * buy_price
                buy_signals.append(data['Close'][i])
                sell_signals.append(np.nan)
            else:
                buy_signals.append(np.nan)
                sell_signals.append(np.nan)
        elif data['Close'][i] > data['Upper Band'][i] and data['ADX'][i] > 20 and data['RSI'][i] > 70:
            if position > 0:  # Sell signal
                sell_price = data['Close'][i]
                cash += position * sell_price
                position = 0
                sell_signals.append(data['Close'][i])
                buy_signals.append(np.nan)
            else:
                buy_signals.append(np.nan)
                sell_signals.append(np.nan)
        else:
            buy_signals.append(np.nan)
            sell_signals.append(np.nan)

        current_value = cash + position * data['Close'][i]
        portfolio_value.append(current_value)

    data['Buy Signal'] = buy_signals
    data['Sell Signal'] = sell_signals
    data['Portfolio Value'] = portfolio_value

    return data, initial_capital, cash + position * data['Close'].iloc[-1]

def buy_and_hold(data, initial_capital=100000):
    buy_price = data['Close'].iloc[0]
    position = initial_capital // buy_price
    cash = initial_capital - position * buy_price
    final_value = cash + position * data['Close'].iloc[-1]
    return initial_capital, final_value

def calculate_dji_performance(start, end, initial_capital=100000):
    dji = yf.download('^DJI', start=start, end=end)
    buy_price = dji['Close'].iloc[0]
    position = initial_capital // buy_price
    cash = initial_capital - position * buy_price
    final_value = cash + position * dji['Close'].iloc[-1]
    return initial_capital, final_value

def analyze_stock(ticker, start_date, end_date, initial_capital=100000):
    # Download data
    data = download_data(ticker, start=start_date, end=end_date)

    # Calculate indicators
    data = calculate_bollinger_bands(data)
    data = calculate_adx(data)
    data = calculate_rsi(data)

    # Apply trading strategy
    data, initial_bollinger, final_bollinger = apply_trading_strategy(data, initial_capital)

    # Apply Buy and Hold strategy
    initial_bh, final_bh = buy_and_hold(data)

    # Calculate DJI performance
    initial_dji, final_dji = calculate_dji_performance(start=start_date, end=end_date)

    # Print results
    print()
    print(f"Bollinger Bands Strategy: Initial = ${initial_bollinger:.2f}, Final = ${final_bollinger:.2f}, Return = ${(final_bollinger - initial_bollinger):.2f}")
    print(f"Buy and Hold Strategy: Initial = ${initial_bh:.2f}, Final = ${final_bh:.2f}, Return = ${(final_bh - initial_bh):.2f}")
    print(f"DJI Investment: Initial = ${initial_dji:.2f}, Final = ${final_dji:.2f}, Return = ${(final_dji - initial_dji):.2f}")

In [17]:
# Example usage
analyze_stock('AAPL', '2020-01-01', '2022-01-01')
analyze_stock('GOOGL', '2020-01-01', '2022-01-01')
analyze_stock('SPY', '2020-01-01', '2022-01-01')

[*********************100%%**********************]  1 of 1 completed
C:\Users\DaComputs\AppData\Local\Temp\ipykernel_8888\1805249860.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data['Close'][i] < data['Lower Band'][i] and data['ADX'][i] > 20 and data['RSI'][i] < 30:
C:\Users\DaComputs\AppData\Local\Temp\ipykernel_8888\1805249860.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif data['Close'][i] > data['Upper Band'][i] and data['ADX'][i] > 20 and data['RSI'][i] > 70:
C:\Users\DaComputs\AppData\Local\Temp\ipykernel_8888\1805249860.py:71: FutureWarning: Series.__getitem__ treating keys as positions i


Bollinger Bands Strategy: Initial = $100000.00, Final = $167157.34, Return = $67157.34
Buy and Hold Strategy: Initial = $100000.00, Final = $236404.22, Return = $136404.22
DJI Investment: Initial = $100000.00, Final = $122408.50, Return = $22408.50

Bollinger Bands Strategy: Initial = $100000.00, Final = $122108.41, Return = $22108.41
Buy and Hold Strategy: Initial = $100000.00, Final = $211646.71, Return = $111646.71
DJI Investment: Initial = $100000.00, Final = $122408.50, Return = $22408.50


[*********************100%%**********************]  1 of 1 completed


Bollinger Bands Strategy: Initial = $100000.00, Final = $156016.63, Return = $56016.63
Buy and Hold Strategy: Initial = $100000.00, Final = $146077.63, Return = $46077.63
DJI Investment: Initial = $100000.00, Final = $122408.50, Return = $22408.50


In [9]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data for the stock
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2020-12-01', end='2021-01-01')

# Print the first few rows of the dataframe
print(tickerDf.head())

# Ensure the index is in datetime format
tickerDf.index = pd.to_datetime(tickerDf.index)

# Calculate the number of years
start_date = tickerDf.index.min()
end_date = tickerDf.index.max()
total_years = (end_date - start_date).days / 365.25

print(f"Total years in dataset: {total_years:.2f} years")


                                 Open        High         Low       Close  \
Date                                                                        
2020-12-01 00:00:00-05:00  118.597529  121.008485  117.617465  120.273438   
2020-12-02 00:00:00-05:00  119.587401  120.910494  118.479932  120.626274   
2020-12-03 00:00:00-05:00  121.057483  121.312302  119.773602  120.489052   
2020-12-04 00:00:00-05:00  120.155834  120.410652  119.097363  119.812813   
2020-12-07 00:00:00-05:00  119.871611  122.086558  119.812810  121.282906   

                              Volume  Dividends  Stock Splits  
Date                                                           
2020-12-01 00:00:00-05:00  127728200        0.0           0.0  
2020-12-02 00:00:00-05:00   89004200        0.0           0.0  
2020-12-03 00:00:00-05:00   78967600        0.0           0.0  
2020-12-04 00:00:00-05:00   78260400        0.0           0.0  
2020-12-07 00:00:00-05:00   86712000        0.0           0.0  
Total years 